In [1]:
import json
import requests
import time
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import datetime
from config import api_key
from config import password

import psycopg2
import sqlite3

In [2]:
conn = psycopg2.connect(database="world_happiness_index", user = "postgres", password = password, host = "127.0.0.1", port = "5432")

In [3]:


country_list = list()


cur = conn.cursor()
cur.execute(f'''SELECT DISTINCT year_2015.country AS YEAR2015, year_2016.country AS YEAR2016, year_2017.country AS YEAR2017, year_2018.country AS YEAR2018, year_2019.country AS YEAR2019
FROM year_2015
JOIN year_2016
ON	year_2015.country = year_2016.country
JOIN year_2017
ON	year_2015.country = year_2017.country
JOIN year_2018
ON	year_2015.country = year_2018.country
JOIN year_2019
ON	year_2015.country = year_2019.country; ''')

rows = cur.fetchall()

for row in rows:
    for item in row:
        country_list.append(item)
    
cur.close()

# removing duplicates from list
country_list = list(dict.fromkeys(country_list))

country_list

['Afghanistan',
 'Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Haiti',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Ivory Coast',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Madagascar'

In [68]:
# testing api pull from google geo locations


#creating api/config information
url = 'https://maps.googleapis.com/maps/api/geocode/json?address='

#build query URL
query_url = url+"peru"+"&key="+api_key

#checking format
# print(query_url)

#testing pull from api
response = requests.get(query_url).json()
# testing index
response["results"][0]["geometry"]["location"]["lng"]

# d = response["results"][0]["geometry"]["location"]

# d

{'lat': -9.189967, 'lng': -75.015152}

In [29]:

#Pulling data via API

#creating empty lists for variables returned from API
lat = []
lng = []
# country = []
country_list_2 =[]

# counter + row tracker 
count = 0
set_num = 1


#looping through data to get lat and lng data for cities
print('Beginning Data Retrieval')
print('-----------------------------')

for country in country_list:
    #adding try logic to code to keep it moving if no result
    try:
        response = requests.get(url+country+"&key="+api_key).json()
        lat.append(response["results"][0]["geometry"]["location"]["lat"])
        lng.append(response["results"][0]["geometry"]["location"]["lng"])
#         country.append(response['sys']['country'])
        
        #if logic to track where we are in data set and eliminate duplicates
        if count >= 49:
            count = 1
            set_num = set_num + 1
            country_list_2.append(country)
            
        else:
            count = count + 1
            country_list_2.append(country)
            
        #displaying where we are in data set and what city was processed
        print(f'Processing record {count} of set {set_num} | {country}')
        #exception value for row if not found
    except Exception:
        print('Country not found. skipping ...')

#confirming looped through all data in set
print('-----------------------------')
print('data retrieval complete')

Beginning Data Retrieval
-----------------------------
Processing record 1 of set 1 | Afghanistan
Processing record 2 of set 1 | Albania
Processing record 3 of set 1 | Algeria
Processing record 4 of set 1 | Argentina
Processing record 5 of set 1 | Armenia
Processing record 6 of set 1 | Australia
Processing record 7 of set 1 | Austria
Processing record 8 of set 1 | Azerbaijan
Processing record 9 of set 1 | Bahrain
Processing record 10 of set 1 | Bangladesh
Processing record 11 of set 1 | Belarus
Processing record 12 of set 1 | Belgium
Processing record 13 of set 1 | Benin
Processing record 14 of set 1 | Bhutan
Processing record 15 of set 1 | Bolivia
Processing record 16 of set 1 | Bosnia and Herzegovina
Processing record 17 of set 1 | Botswana
Processing record 18 of set 1 | Brazil
Processing record 19 of set 1 | Bulgaria
Processing record 20 of set 1 | Burkina Faso
Processing record 21 of set 1 | Burundi
Processing record 22 of set 1 | Cambodia
Processing record 23 of set 1 | Cameroon


In [31]:

#creating dictionary with data pulled from api
country_dict={"country": country_list_2,
              "lat": lat,
              "lng": lng}

country_df= pd.DataFrame(country_dict)

 
print(country_df.shape)
country_df.head()

(146, 3)


,country,lat,lng
0,Afghanistan,33.939110,67.709953
1,Albania,41.153332,20.168331
2,Algeria,28.033886,1.659626
3,Argentina,-38.416097,-63.616672
4,Armenia,40.069099,45.038189


In [74]:
country_df.to_csv('location_data.csv')